In [13]:
import os
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.utils import resample
import joblib
import random

# Fonctions d'augmentation des données
def augment_image(img):
    # Choisir aléatoirement d'appliquer l'augmentation ou non
    if random.choice([True, False]):
        # Retournement aléatoire
        img = cv2.flip(img, 1)
    if random.choice([True, False]):
        # Rotation aléatoire
        angle = random.randint(-15, 15)
        center = (img.shape[1] // 2, img.shape[0] // 2)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        img = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))
    if random.choice([True, False]):
        # bruit aléatoire
        noise = np.random.normal(0, 0.1, img.shape)
        img = img + noise * 255
        img = np.clip(img, 0, 255).astype(np.uint8)
    return img

def load_images_from_folder(folder, img_size=(48, 48), limit=None, augment=False):
    images = []
    labels = []
    count = 0
    for subfolder in os.listdir(folder):
        label = subfolder
        subfolder_path = os.path.join(folder, subfolder)
        if os.path.isdir(subfolder_path):
            for filename in os.listdir(subfolder_path):
                if limit and count >= limit:
                    break
                img_path = os.path.join(subfolder_path, filename)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is not None:
                    img_resized = cv2.resize(img, img_size)
                    if augment:
                        img_resized = augment_image(img_resized)
                    images.append(img_resized)
                    labels.append(label)
                    count += 1
                else:
                    print(f"Erreur lors du chargement de l'image: {img_path}")
            if limit and count >= limit:
                break
    return np.array(images), np.array(labels)

# Chargement et prétraitement des images
train_folder = '/home/lionel/anaconda3/M1_IA_AVANCE/EMOTION_WEBCAM/dataset'
subset_limit = 10000  
images, labels = load_images_from_folder(train_folder, limit=subset_limit, augment=True)

In [15]:
# Normalisation et réduction de dimension avec PCA
scaler = StandardScaler()
pca = PCA(n_components=100)  # Réduction des dimensions à 100 pour accélérer le traitement
le = LabelEncoder()

if len(images.shape) == 3:
    n_samples, width, height = images.shape
    X = images.reshape(n_samples, width * height)
else:
    X = images

X = scaler.fit_transform(X)
X_pca = pca.fit_transform(X)

# Encodage des labels
y = le.fit_transform(labels)


In [17]:
# Équilibrer l'ensemble de données (suréchantillonner les classes minoritaires)
unique_classes = np.unique(y)
X_balanced, y_balanced = [], []

for cls in unique_classes:
    X_cls = X_pca[y == cls]
    y_cls = y[y == cls]
    if len(X_cls) < len(X_pca) // len(unique_classes):
        X_upsampled, y_upsampled = resample(X_cls, y_cls, n_samples=len(X_pca) // len(unique_classes), random_state=42)
    else:
        X_upsampled, y_upsampled = X_cls, y_cls
    X_balanced.append(X_upsampled)
    y_balanced.append(y_upsampled)

X_balanced = np.vstack(X_balanced)
y_balanced = np.hstack(y_balanced)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)

# GridSearch pour trouver les meilleurs hyperparamètres
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [0.01, 0.001, 0.0001],
    'kernel': ['rbf', 'poly', 'sigmoid']
}

grid_search_svm = GridSearchCV(SVC(), param_grid, refit=True, verbose=3, cv=3, n_jobs=-1)
grid_search_svm.fit(X_train, y_train)

# Utilisation du meilleur modèle trouvé
best_svm_model = grid_search_svm.best_estimator_
best_svm_model.fit(X_train, y_train)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


SVC(C=10, gamma=0.001)

In [21]:
y_pred = best_svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy finale avec SVM: {accuracy * 100:.2f}%')

model_file = "final_emotion_detection_svm_model.joblib"
label_encoder_file = "label_encoder.joblib"
pca_file = "pca.joblib"

joblib.dump(best_svm_model, model_file)
joblib.dump(le, label_encoder_file)
joblib.dump(pca, pca_file)

print(f'Model final SVM sauvegardé dans {model_file}')
print(f'Label encoder sauvegardé dans {label_encoder_file}')
print(f'PCA sauvegardé dans {pca_file}')


Accuracy finale avec SVM: 69.88%
Model final SVM sauvegardé dans final_emotion_detection_svm_model.joblib
Label encoder sauvegardé dans label_encoder.joblib
PCA sauvegardé dans pca.joblib
